In [12]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd


In [14]:
excel_file = r"vi_prod_class_22-10.xlsx"  # Đường dẫn đến file Excel của bạn

In [ ]:
df_1 = pd.read_excel(excel_file, sheet_name = r'tranformed Kqxd trước mã số', skiprows=1).iloc[:,:3].fillna('')
df_2 = pd.read_excel(excel_file, sheet_name = r'transformed Phân loại hàng hóa ', skiprows=1).iloc[:,:3].fillna('')

df = pd.concat([df_1,df_2])
df.columns = ['link','doc_no','doc_date']
df = df.drop_duplicates()
df['doc_name'] = df['doc_no'].replace('/', '_', regex=True).replace('-', '_', regex=True)+'_'+df['doc_date'].replace('-', '_', regex=True)


In [44]:
df.head()

,link,doc_no,doc_date,doc_name
0,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,3664/TB-TCHQ,29-07-2024,3664_TB_TCHQ_29_07_2024
1,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,3349/TB-TCHQ,10-07-2024,3349_TB_TCHQ_10_07_2024
2,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,3313/TB-TCHQ,08-07-2024,3313_TB_TCHQ_08_07_2024
3,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,3312/TB-TCHQ,08-07-2024,3312_TB_TCHQ_08_07_2024
4,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,2710/TB-TCHQ,12-06-2024,2710_TB_TCHQ_12_06_2024


In [105]:
df.to_excel("link_for_download_pdf.xlsx",index = True)

In [102]:
# Đường dẫn để đăng nhập
login_url = 'https://thuvienphapluat.vn/page/ajaxcontroler.aspx'

# Dữ liệu cần gửi để đăng nhập
payload = {
    'l_txtUser': 'UNICUSVN',
    'l_txtPass': 'UNIUNI',
    'action': 'Login'
}

# Headers cần thiết cho yêu cầu
headers = {
    'Accept': '*/*',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'no-cache',
    'Content-Type': 'application/x-www-form-urlencoded',
    'DNT': '1',
    'Origin': 'https://thuvienphapluat.vn',
    'Pragma': 'no-cache',
    'Referer': 'https://thuvienphapluat.vn/page/login.aspx?',
    'Sec-CH-UA': '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
    'Sec-CH-UA-Mobile': '?0',
    'Sec-CH-UA-Platform': '"Windows"',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest'
}

# Khởi tạo session để giữ cookie
session = requests.Session()

# Gửi yêu cầu POST để đăng nhập
login_response = session.post(login_url, data=payload, headers=headers)
# Kiểm tra đăng nhập thành công
if login_response.ok:
    print("Đăng nhập thành công!")
else:
    print("Đăng nhập thất bại:", login_response.status_code)


Đăng nhập thành công!


In [99]:
def get_downloadlink(link):
    response = session.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    find_pdf_link = soup.find('div',id = 'tab8').find('a',id ='ctl00_Content_ThongTinVB_pdfHyperLink')
    if find_pdf_link:
        all_script = soup.find_all('script')
        pattern = r'/documents/download\.aspx\?id=[\w%]+'


        for idx,text in enumerate(all_script):
            if '__urldl' in text.get_text():
                matches = re.findall(pattern, text.get_text())
                if matches:
                    print(matches)

        matches[-1]
        download_url = f'https://thuvienphapluat.vn{matches[-1]}&part=-0'
        print(download_url)
        return download_url
    else:
        print("không có link pdf")

In [ ]:
def download_pdf(download_link,file_name,idx):
    path = r"D:\Onedrive\OneDrive - UNI CONSULTING\Eximhow_pdf\\"
    # Gửi yêu cầu tải file PDF
    response = session.get(download_link, headers=headers, stream=True)

    # Kiểm tra nếu yêu cầu thành công và lưu file
    if response.ok:
        
        with open(f"{path}{file_name}{}.pdf", "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"Tải file thành công và lưu thành {file_name}.pdf")
    else:
        print("Lỗi khi tải file:", response.status_code)


In [91]:
df.iloc[10:20,:]

,link,doc_no,doc_date,doc_name
10,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,1970/TB-TCHQ,08-05-2024,1970_TB_TCHQ_08_05_2024
11,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,1395/TB-TCHQ,02-04-2024,1395_TB_TCHQ_02_04_2024
12,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,191/TB-TCHQ,12-01-2024,191_TB_TCHQ_12_01_2024
13,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,143/TB-TCHQ,10-01-2024,143_TB_TCHQ_10_01_2024
14,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,144/TB-TCHQ,10-01-2024,144_TB_TCHQ_10_01_2024
15,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,64/TB-TCHQ,04-01-2024,64_TB_TCHQ_04_01_2024
16,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,6201/TB-TCHQ,01-12-2023,6201_TB_TCHQ_01_12_2023
17,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,6200/TB-TCHQ,01-12-2023,6200_TB_TCHQ_01_12_2023
18,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,5751/TB-TCHQ,08-11-2023,5751_TB_TCHQ_08_11_2023
19,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,4945/TB-TCHQ,22-09-2023,4945_TB_TCHQ_22_09_2023


In [96]:
for idx,row in df.iloc[10:20,:].iterrows():
    print (idx)
    print(row[0])
    download_link= get_downloadlink(row[0])
    print(row[3])
    file_name = row[3]
    download_pdf(download_link,file_name)

10
https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-1970-TB-TCHQ-2024-ket-qua-xac-dinh-truoc-ma-so-doi-voi-SPEED-SENSOR-ASSY-610283.aspx


C:\Users\nguye\AppData\Local\Temp\ipykernel_21292\683674137.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(row[0])
C:\Users\nguye\AppData\Local\Temp\ipykernel_21292\683674137.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  download_link= get_downloadlink(row[0])


51
['/documents/download.aspx?id=KrnOQruGxkbQVRjg%2bmoe8g%3d%3d']
https://thuvienphapluat.vn/documents/download.aspx?id=KrnOQruGxkbQVRjg%2bmoe8g%3d%3d&part=-0
1970_TB_TCHQ_08_05_2024
Tải file thành công và lưu thành downloaded_file.pdf
11
https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-1395-TB-TCHQ-2024-ket-qua-xac-dinh-truoc-ma-so-Green-Energy-607239.aspx


C:\Users\nguye\AppData\Local\Temp\ipykernel_21292\683674137.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(row[3])
C:\Users\nguye\AppData\Local\Temp\ipykernel_21292\683674137.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  file_name = row[3]


51
['/documents/download.aspx?id=0rCbuO0Rpgrcl5G3dK2Clg%3d%3d']
https://thuvienphapluat.vn/documents/download.aspx?id=0rCbuO0Rpgrcl5G3dK2Clg%3d%3d&part=-0
1395_TB_TCHQ_02_04_2024
Tải file thành công và lưu thành downloaded_file.pdf
12
https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-191-TB-TCHQ-2024-ket-qua-xac-dinh-truoc-ma-so-Glentaz-Forte-596694.aspx
51
['/documents/download.aspx?id=exAFs0zAVSokJLDh6bfnYQ%3d%3d']
https://thuvienphapluat.vn/documents/download.aspx?id=exAFs0zAVSokJLDh6bfnYQ%3d%3d&part=-0
191_TB_TCHQ_12_01_2024
Tải file thành công và lưu thành downloaded_file.pdf
13
https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-143-TB-TCHQ-2024-ket-qua-xac-dinh-truoc-ma-so-KLENZIT-MS-596867.aspx
51
['/documents/download.aspx?id=hEOaeYaowYzhqBWvqHioZQ%3d%3d']
https://thuvienphapluat.vn/documents/download.aspx?id=hEOaeYaowYzhqBWvqHioZQ%3d%3d&part=-0
143_TB_TCHQ_10_01_2024
Tải file thành công và lưu thành downloaded_file.pdf
14
https://thuvienphapluat.vn/van-ban/X

In [100]:
test = get_downloadlink('https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-1970-TB-TCHQ-2024-ket-qua-xac-dinh-truoc-ma-so-doi-voi-SPEED-SENSOR-ASSY-610283.aspx')

['/documents/download.aspx?id=KrnOQruGxkbQVRjg%2bmoe8g%3d%3d']
https://thuvienphapluat.vn/documents/download.aspx?id=KrnOQruGxkbQVRjg%2bmoe8g%3d%3d&part=-0


In [104]:
download_pdf(test,"test")

Tải file thành công và lưu thành test.pdf
